In [1]:
import pandas as pd
import numpy as np
import openpyxl

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

import ast
from sklearn.preprocessing import MultiLabelBinarizer

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
df = pd.read_csv('spotify master.csv')
df.shape

(460945, 2114)

In [3]:
miss = pd.DataFrame(df.isnull().sum().sort_values(ascending=False), columns=['missing'])
miss['%'] = miss['missing']/df.shape[0]
miss.head(50)

,missing,%
album label,21,0.000046
artist number of followers,11,0.000024
el paso indie,0,0.000000
edm,0,0.000000
ecuadorian pop,0,0.000000
ectofolk,0,0.000000
ecm-style jazz,0,0.000000
ebm,0,0.000000
eau claire indie,0,0.000000
zouk,0,0.000000


In [4]:
df.fillna({'album label':'no info', 'artist number of followers':0}, inplace = True)

In [32]:
df['album label'].unique()

array(['Parlophone UK', 'Grandom records', 'Arista/LaFace Records', ...,
       'QZDPF1814528', 'Revolution Records', 'Tamarindo Rekordsz'],
      dtype=object)

In [8]:
df['artist genres'] = df['artist genres'].apply(lambda s: list(ast.literal_eval(s)))

# This package allows the creation of dummies that are not mutually exclusive
# A given song can belong to multiple genres
mlb = MultiLabelBinarizer()
artist_genres_dummies = pd.DataFrame(mlb.fit_transform(df['artist genres']),columns=mlb.classes_, index=df.index)

raw = pd.concat([df,artist_genres_dummies], axis = 1)

list_of_all_columns = list(raw.columns)
list_of_genre_columns = list(artist_genres_dummies.columns)
list_of_baseline_columns = [col for col in list_of_all_columns if col not in list_of_genre_columns]


In [11]:
list_of_all_columns = list(df.columns)
list_of_genre_columns = list(artist_genres_dummies.columns)
list_of_baseline_columns = [col for col in list_of_all_columns if col not in list_of_genre_columns]

check_dup_columns = list_of_baseline_columns.copy()
check_dup_columns.remove(check_dup_columns[0])
check_dup_columns.remove('song_id')
check_dup_columns.remove('date of data pull')
check_dup_columns.remove('artist genres')

for col in check_dup_columns:
    baseline = df.groupby(['song_id'], as_index = False).agg({'date of data pull':'min'})
    check = df.groupby(['song_id', col], as_index = False).agg({'date of data pull':'min'})
    logging.debug(f'when adding {col} there are {check.shape[0]-baseline.shape[0]} extra rows')

DEBUG:root:when adding song name there are 130 extra rows
DEBUG:root:when adding album there are 87 extra rows
DEBUG:root:when adding artist there are 40 extra rows
DEBUG:root:when adding artist popularity there are 45169 extra rows
DEBUG:root:when adding artist number of followers there are 355587 extra rows
DEBUG:root:when adding artist type there are 0 extra rows
DEBUG:root:when adding album label there are 365 extra rows
DEBUG:root:when adding album popularity there are 111789 extra rows
DEBUG:root:when adding song release date there are 180 extra rows
DEBUG:root:when adding length there are 37 extra rows
DEBUG:root:when adding song popularity there are 59640 extra rows
DEBUG:root:when adding key there are 45 extra rows
DEBUG:root:when adding mode there are 22 extra rows
DEBUG:root:when adding acousticness there are 430 extra rows
DEBUG:root:when adding valence there are 680 extra rows
DEBUG:root:when adding danceability there are 441 extra rows
DEBUG:root:when adding energy there 

### Checkout duplication with the same song being in multiple albums
- You can have songs in multiple albums. Released the same data (1 album is the generic and another album is the collector's edition)
- There are songe named 17, 2009 etc. We might want to remove those
- Let's create a composite name-album index and continue investigating

In [9]:
test = df[list_of_baseline_columns]

In [14]:
test['check'] = test['song name'] + test['album']
test.head()

<ipython-input-14-23a7d5cc8054>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['check'] = test['song name'] + test['album']


,Unnamed: 0,date of data pull,song_id,song name,album,artist,artist genres,artist popularity,artist number of followers,artist type,...,tamborazo,thai hip hop,touhou,tunantada,turkish deep house,vintage italian pop,virginia indie,weirdcore,wu fam,check
0,0,2021-04-05,3AJwUDP919kvQ9QcozQPxg,Yellow,Parachutes,Coldplay,"[permanent wave, pop]",89,29604326.0,artist,...,0,0,0,0,0,0,0,0,0,YellowParachutes
1,1,2021-04-05,0QXEvVJyHIQhVzci6kBULo,Year 2000,Fear,Matias Grandom,[],32,29.0,artist,...,0,0,0,0,0,0,0,0,0,Year 2000Fear
2,2,2021-04-05,0I3q5fE6wg7LIfHGngUTnV,Ms. Jackson,Stankonia,Outkast,"[atl hip hop, dirty south rap, hip hop, rap, s...",78,1616587.0,artist,...,0,0,0,0,0,0,0,0,0,Ms. JacksonStankonia
3,3,2021-04-05,6pM25DLzJb5oWj74d3ElXI,2000 Light Years Away,Kerplunk!,Green Day,"[permanent wave, pop punk, punk]",84,11947151.0,artist,...,0,0,0,0,0,0,0,0,0,2000 Light Years AwayKerplunk!
4,4,2021-04-05,3yfqSUWxFvZELEM4PmlwIR,The Real Slim Shady,The Marshall Mathers LP,Eminem,"[detroit hip hop, hip hop, rap]",94,43216118.0,artist,...,0,0,0,0,0,0,0,0,0,The Real Slim ShadyThe Marshall Mathers LP


In [15]:
dupped = test.groupby(['song name', 'album'], as_index = False).agg({'date of data pull':'min'}) 
dedupped = dupped.groupby(['song name'], as_index = False).agg({'date of data pull':'count'}).rename({'date of data pull':'row count'},axis = 1 )


list_of_duplicate_names = dedupped[dedupped['row count']>1]['song name']

check = test[test['song name'].isin(list_of_duplicate_names)]
check.to_excel('investigate.xlsx')

In [16]:
dupped = test.groupby(['song_id', 'check'], as_index = False).agg({'date of data pull':'min'}) 
dedupped = dupped.groupby(['song_id'], as_index = False).agg({'date of data pull':'count'}).rename({'date of data pull':'row count'},axis = 1 )


list_of_duplicate_names = dedupped[dedupped['row count']>1]['song_id']

check = test[test['song_id'].isin(list_of_duplicate_names)]
check.to_excel('investigate.xlsx')

### Checkout duplication same song having multiple musical characteristics

In [6]:
# create a composite index
df["name_album"] = df["song name"] + df["album"]

dupped = df.groupby(['name_album', 'length'], as_index = False).agg({'date of data pull':'min'}) 
dedupped = dupped.groupby(['name_album'], as_index = False).agg({'date of data pull':'count'}).rename({'date of data pull':'row count'},axis = 1 )

list_of_duplicate_names = dedupped[dedupped['row count']>1]['name_album']

check = df[df['name_album'].isin(list_of_duplicate_names)]
check.to_excel('investigate.xlsx')


## Checkout Spread of continuous variables

In [10]:
list_of_continuous_vars = [
'artist popularity'
,'artist number of followers'
,'album popularity'
,'length'
,'song popularity'
,'key'
,'mode'
,'acousticness'
,'valence'
,'danceability'
,'energy'
,'instrumentalness'
,'liveness'
,'loudness'
,'speechiness'
,'tempo'
,'time signature'
,'tempo confidence'
,'key confidence'
,'time signature confidence'
,'mode confidence'
,'rhythm version'
,'synch version'
,'number of segments'
,'number of bars'
,'number of beats'
,'number of sections'
,'number of tatums'
]


test.describe(percentiles=np.linspace(0,1,101)).to_csv('check.csv')

# for var_ in list_of_continuous_vars:
#     test[var_].describe

In [20]:
dupped = test.groupby(['song_id', 'key'], as_index = False).agg({'date of data pull':'min'}) 
dedupped = dupped.groupby(['song_id'], as_index = False).agg({'date of data pull':'count'}).rename({'date of data pull':'row count'},axis = 1 )


list_of_duplicate_names = dedupped[dedupped['row count']>1]['song_id']

check = test[test['song_id'].isin(list_of_duplicate_names)]
check.sort_values(by=['song_id', 'date of data pull'], inplace = True)
check.to_excel('investigate.xlsx')

<ipython-input-20-ef3a6e1f9342>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  check.sort_values(by=['song_id', 'date of data pull'], inplace = True)


In [21]:
check = test[test['number of bars']==0]

list_of_songs_of_interest = list(check['song_id'].unique())

exp = test[test['song_id'].isin(list_of_songs_of_interest)]

exp.sort_values(by=['song_id', 'date of data pull'], inplace = True)
exp.to_excel('investigate.xlsx')

<ipython-input-21-4d712294ea40>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp.sort_values(by=['song_id', 'date of data pull'], inplace = True)


In [24]:
std_vars = ['artist popularity', 'artist number of followers', 'album popularity', 'song popularity']
std_vars_2 = []

for var_ in std_vars:
    test[f'{var_} std'] = test.groupby('song_id', as_index = False)[var_].transform(lambda s: s.std())
    std_vars_2.append(f'{var_} std')

test[std_vars_2].describe(percentiles=np.linspace(0,1,101)).to_excel('investigate.xlsx')

<ipython-input-24-6d9c50bb2c2b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[f'{var_} std'] = test.groupby('song_id', as_index = False)[var_].transform(lambda s: s.std())


In [30]:
exp = test[test['song popularity std']>2]

exp.sort_values(by=['song_id', 'date of data pull'], inplace = True)
exp.to_excel('investigate.xlsx')

<ipython-input-30-09ad76e2f9f3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp.sort_values(by=['song_id', 'date of data pull'], inplace = True)


- Artist number of followers > 0 
- If album popularity == 0 AND STD within the song ID > a constant, DELETE
- If song popularity == 0 AND STD within the song ID > a constant, DELETE
- key, mode take the latest ones!
- acousticness, valence, danceability the zeros are legit
- rhythm version, synch version should be dropped. They are non-variables
- if confidence = 0 drop
- Artist popularity: Shave 5%
- Song popularity if zero and > 10%
- song popularity if greater that 1%
- Same for Album opularity

In [49]:
pct = test['artist popularity'].describe(percentiles=np.linspace(0,1,101))
pct
pct[94]

# 103 for 99th pctl
# 99 for 95th pctl
# 94 for 90th pctl

count    460945.000000
mean         55.906457
std          28.543667
min           0.000000
0%            0.000000
             ...      
97%          95.000000
98%          96.000000
99%          98.000000
100%        100.000000
max         100.000000
Name: artist popularity, Length: 106, dtype: float64

88.0

## GENERAL

- How many events do we have?
- are event ID's unique?

In [2]:
number_of_unique_event_ids = df['event id'].nunique()
number_of_rows = df.shape[0]
number_of_unique_events = df['event name'].nunique()

if number_of_unique_event_ids == number_of_rows:
    print('The data set has one row per event id')
else:
    print('The data set has', number_of_rows, 'rows but only',  number_of_unique_event_ids, 'unique event ids')

pd.set_option('display.max_rows',120)

The data set has 140 rows but only 139 unique event ids


## DUPLICATION

- Events with the same ID keep on reappearing with different information
- What changes?

In [3]:
# Create a dedupped data set with a row count of the duplicate rows
check_duplicates = df[['event id','info']].groupby(['event id'], as_index = False).count().rename({'info':'duplication count'}, axis = 1)

# Put the duplicate event ids in a list
list_of_duplicate_event_ids = list(check_duplicates[check_duplicates['duplication count']>1]['event id'])

# Add the duplication counter and export the duplicate rows for review
duplicate_rows = df[df['event id'].isin(list_of_duplicate_event_ids)]

duplicate_rows_with_dup_counter = pd.merge(
    duplicate_rows, check_duplicates
    ,how = 'inner'
    ,left_on = 'event id'
    ,right_on = 'event id'
    )   

# duplicate_rows_with_dup_counter.to_excel('~/downloads/check ticketmaster duplication.xlsx')

## EDA of Caregorical Variables

- Are certain venues more prolific?
- Breakout of event types. Tabs and Crosstabs 
- How is ticket limit looking?

In [40]:
freq = df.groupby('venue name').agg({'info':'count'}).rename({'info':'row count'}, axis = 1)
freq['%'] = freq['row count']/df.shape[0]
freq


for type_var in ['segment', 'genre', 'subGenre', 'type', 'subType', 'ticket limit']:

    freq = df.groupby(type_var).agg({'info':'count'}).rename({'info':'row count'}, axis = 1)
    freq['%'] = freq['row count']/df.shape[0]
    freq
    

freq = pd.crosstab(df['segment'], df['genre'])
freq

freq = pd.crosstab(df['segment'], df['subGenre'])
freq

freq = pd.crosstab(df['genre'], df['subGenre'])
freq


,row count,%
venue name,,
Carolina Theatre,4,0.029851
Coastal Credit Union Music Park at Walnut Creek,0,0.000000
DPAC - Durham Performing Arts Center,60,0.447761
Duke Energy Center for the Performing Arts,0,0.000000
Florence Center,0,0.000000
PNC Arena,2,0.014925
Raleigh Improv,0,0.000000
Red Hat Amphitheater,0,0.000000
Steven Tanger Center for the Performing Arts,20,0.149254


,row count,%
segment,,
Arts & Theatre,67,0.500000
Miscellaneous,13,0.097015
Music,4,0.029851
Sports,2,0.014925
Undefined,0,0.000000


,row count,%
genre,,
Classical,5,0.037313
Comedy,2,0.014925
Country,1,0.007463
Hockey,2,0.014925
Jazz,0,0.000000
Opera,0,0.000000
Other,0,0.000000
R&B,1,0.007463
Rock,2,0.014925


,row count,%
subGenre,,
Bluegrass,1,0.007463
Comedy,2,0.014925
Jazz,0,0.000000
Musical,60,0.447761
NHL,2,0.014925
Opera,0,0.000000
Other,0,0.000000
Pop,2,0.014925
R&B,1,0.007463


,row count,%
type,,
Event Style,0,0.000000
Group,2,0.014925
Merchandise,0,0.000000
Undefined,71,0.529851
Upsell,13,0.097015


,row count,%
subType,,
Competition,0,0.000000
Gift Certificate,0,0.000000
Special Entry,13,0.097015
Team,2,0.014925
Undefined,71,0.529851


,row count,%
ticket limit,,
OFF,0,0.000000
There is an overall 6 ticket limit for this event.,1,0.007463
There is an overall 8 ticket limit for this event.,63,0.470149


genre,Classical,Comedy,Country,Hockey,Jazz,Opera,Other,R&B,Rock,Theatre,Undefined
segment,,,,,,,,,,,
Arts & Theatre,7,31,0,0,0,1,0,0,0,68,0
Miscellaneous,0,0,0,0,0,0,0,0,0,0,15
Music,0,0,1,0,1,0,2,1,4,0,0
Sports,0,0,0,2,0,0,0,0,0,0,0
Undefined,0,0,0,0,0,0,0,0,0,0,1


subGenre,Bluegrass,Comedy,Jazz,Musical,NHL,Opera,Other,Pop,R&B,Symphonic,Undefined
segment,,,,,,,,,,,
Arts & Theatre,0,31,0,68,0,1,0,0,0,7,0
Miscellaneous,0,0,0,0,0,0,0,0,0,0,15
Music,1,0,1,0,0,0,2,4,1,0,0
Sports,0,0,0,0,2,0,0,0,0,0,0
Undefined,0,0,0,0,0,0,0,0,0,0,1


subGenre,Bluegrass,Comedy,Jazz,Musical,NHL,Opera,Other,Pop,R&B,Symphonic,Undefined
genre,,,,,,,,,,,
Classical,0,0,0,0,0,0,0,0,0,7,0
Comedy,0,31,0,0,0,0,0,0,0,0,0
Country,1,0,0,0,0,0,0,0,0,0,0
Hockey,0,0,0,0,2,0,0,0,0,0,0
Jazz,0,0,1,0,0,0,0,0,0,0,0
Opera,0,0,0,0,0,1,0,0,0,0,0
Other,0,0,0,0,0,0,2,0,0,0,0
R&B,0,0,0,0,0,0,0,0,1,0,0
Rock,0,0,0,0,0,0,0,4,0,0,0


## EDA of Caregorical Variables

- What is the range of the date variables?
- Are there missing event start dates?
- price ranges (outliers, etc)

## Are prices moving? (TBD)